In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from pandasql import sqldf
import random
import os
from numba import jit,njit

test_num = 12

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

&nbsp;

## 입력 shape 및 형태 정의 함수

In [2]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [3]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [4]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0.2, mlp_dropout=0.2):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [5]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=30)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [6]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.002, verbose = 1):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.3,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:1'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [7]:
@jit
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

@jit
def y_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(14, len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values - df.iloc[index-1])

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [8]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1000
batch = 64
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

&nbsp;

## Train 과정

In [9]:
losses = []
scalers = {}

for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)
    
    # nan 처리
    df = df.fillna(0)
    # df = df.interpolate(method = 'values')
    
    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']
    scaler = MinMaxScaler()
    x = pd.DataFrame(scaler.fit_transform(x))
    scalers[df_number] = scaler

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    # y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = y_time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y
    
    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch, verbose = 0)
    
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')
    loss = transformer.evaluate(astype_data(x_val), y_val, verbose=0)
    print('\nvalid_loss: ', loss)
    losses.append(loss)

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

  0%|          | 0/37 [00:00<?, ?it/s]

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-09-15 23:15:45.196950: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-15 23:15:45.197052: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-09-15 23:15:45.995590: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-15 23:15:46.156010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1679052.62500, saving model to ./check/transformer-4-1000-64.h5

Epoch 2: val_loss improved from 1679052.62500 to 1679025.50000, saving model to ./check/transformer-4-1000-64.h5


2022-09-15 23:15:46.428297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...


valid_loss:  9172710.0


2022-09-15 23:58:21.575036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 13615087.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 2: val_loss improved from 13615087.00000 to 13615050.00000, saving model to ./check/transformer-27-1000-64.h5


2022-09-15 23:58:22.223235: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 3: val_loss improved from 13615050.00000 to 13615005.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 4: val_loss improved from 13615005.00000 to 13614944.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 5: val_loss improved from 13614944.00000 to 13614868.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 6: val_loss improved from 13614868.00000 to 13614769.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 7: val_loss improved from 13614769.00000 to 13614650.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 8: val_loss improved from 13614650.00000 to 13614503.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 9: val_loss improved from 13614503.00000 to 13614328.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 10: val_loss improved from 13614328.00000 to 13614121.00000, saving model to ./check/transformer-27-1000-64.h5

Epoch 11: val_loss improved from 13614121.00000 to 13613881.00

&nbsp;

## Test 과정

In [ ]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)


        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]

        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)
        # df = df.interpolate(method = 'values')

        df = pd.DataFrame(scalers[file_number].transform(df))

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists(f'./model_output/{test_num}') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 41ms/step


2022-09-15 23:11:19.666712: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 37ms/step


2022-09-15 23:11:19.767539: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-15 23:11:59.954734: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 37ms/step


2022-09-15 23:12:00.055900: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 36ms/step


2022-09-15 23:12:00.156993: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 10/10 [00:40<00:00,  4.06s/it]


&nbsp;

## 정답 제출 파일생성

In [ ]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  8.58it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [ ]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.125000,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,-933.801640,-376.154540,-807.707760,-662.516850,-627.800500,-1267.867000,25.135101,-679.149400,-1087.881800,-338.602080,...,-1699.713500,-223.212900,-279.848750,-346.379760,0.059408,-1409.978000,-1122.782100,-977.298300,-669.334300,-544.732600
2,-1482.461200,-531.694700,-1146.115000,-1067.379200,-995.694700,-1649.646500,16.500776,-1146.696500,-1767.954000,-555.215200,...,-1962.611900,-348.852360,-442.324130,-528.893070,-0.029500,-2152.722000,-1525.749800,-1225.155000,-1029.026600,-782.405100
3,-1332.289000,-507.614260,-1025.812000,-978.647640,-927.943050,-1623.616700,11.745609,-1219.098900,-1679.856900,-497.398620,...,-1907.116600,-308.089000,-365.855960,-470.418100,-0.089827,-2054.799000,-1565.210100,-1201.882600,-956.433840,-700.199340
4,-1284.314800,-484.841920,-847.557200,-909.537700,-896.624900,-1577.173600,11.428378,-1142.545900,-1560.664800,-396.609130,...,-1920.123800,-264.539250,-351.245760,-410.230380,0.152426,-2083.172400,-1489.842400,-1166.310000,-946.739750,-661.860840
5,-1345.544600,-461.050020,-781.240660,-906.698100,-920.065400,-1519.441500,11.304709,-1018.175300,-1576.139600,-378.423550,...,-1940.036900,-253.913300,-386.741600,-375.722440,0.005152,-2184.148200,-1460.963500,-1092.884500,-991.653800,-718.463600
6,-817.062130,-289.075930,-409.667850,-530.336900,-556.069100,-1087.164200,-6.808016,-591.675350,-874.537660,-136.599320,...,-1539.610800,-159.389240,-209.771420,-171.167110,0.056735,-1415.321900,-1003.452800,-740.217600,-642.059900,-456.092800
7,-70.949620,-32.725388,-61.191715,-98.020200,-95.925580,-141.851150,-19.307043,-148.192430,-136.692800,-40.852715,...,-365.386200,-30.070559,-39.798800,-40.676662,0.101746,-179.690400,-74.080520,-174.017120,-128.242810,-51.987510
8,-866.597500,-309.295380,-823.044300,-651.986600,-644.017300,-1082.981000,-20.715479,-645.680200,-1108.387100,-358.280760,...,-1352.651400,-214.538500,-291.528660,-361.817440,0.071370,-1450.521500,-1052.335000,-773.171200,-692.731600,-540.878200
9,-1341.819600,-451.646270,-1140.319800,-994.458070,-999.315400,-1447.140000,-16.233881,-997.205200,-1748.104200,-566.546400,...,-1612.674700,-325.238620,-439.665470,-532.948550,-0.052696,-2114.034700,-1475.812000,-988.586800,-989.306900,-757.815900


- 변동률 계산 

In [ ]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      # ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1])/base_d)

    # ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t
    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean())/base_d
    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [ ]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [ ]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,-0.003206,-0.005402,-0.004850,-0.003621,-0.003263,-0.001663,-0.000315,-0.003737,-0.003293,-0.010025,...,-0.007796,-0.008307,0.000006,-0.002952,-0.001875,-0.002722,-0.003897,-0.006194,0,d+1
1,-0.002290,-0.005933,-0.004775,-0.003966,-0.003514,-0.001338,-0.000207,-0.003304,-0.003248,-0.011719,...,-0.008756,-0.009856,-0.000061,-0.002158,-0.001093,-0.001868,-0.003034,-0.004661,0,d+2
2,-0.003059,-0.005796,-0.005523,-0.004253,-0.003955,-0.001333,-0.000213,-0.003329,-0.003748,-0.014352,...,-0.010297,-0.011168,0.000038,-0.002509,-0.000962,-0.001748,-0.003648,-0.004992,0,d+3
3,-0.001707,-0.002963,-0.002560,-0.002330,-0.002160,-0.000771,-0.000193,-0.001482,-0.002444,-0.007985,...,-0.005431,-0.005148,-0.000062,-0.002424,-0.000102,-0.000578,-0.001963,-0.000562,0,d+4
4,0.000097,-0.000524,-0.000100,-0.001118,-0.000891,-0.000309,-0.000224,-0.000085,-0.000800,-0.000701,...,-0.001626,0.000105,0.000057,-0.001904,0.000648,0.000138,-0.000700,0.002321,0,d+5
5,0.000603,-0.000270,0.001473,-0.000897,-0.000625,-0.000170,-0.000170,0.000832,-0.000162,0.001375,...,-0.000864,0.001837,-0.000011,-0.001444,0.001260,0.000321,-0.000238,0.003326,0,d+6
6,0.000395,-0.000596,0.000125,-0.000716,-0.000599,-0.000267,-0.000246,0.001881,-0.000624,-0.001552,...,-0.001939,0.001111,0.000009,-0.001541,0.001227,0.000301,-0.000672,0.002556,0,d+7
7,-0.001102,-0.002267,-0.003267,-0.001837,-0.001932,-0.000948,-0.000259,0.000521,-0.002493,-0.009078,...,-0.006822,-0.004513,-0.000083,-0.003000,-0.000122,-0.000524,-0.002281,-0.001136,0,d+8
8,-0.001076,-0.002605,-0.003132,-0.002245,-0.002356,-0.000814,-0.000191,0.000121,-0.002886,-0.010558,...,-0.007328,-0.005575,-0.000106,-0.002549,0.000249,-0.000256,-0.002474,-0.001117,0,d+9
9,-0.001616,-0.002862,-0.003796,-0.002461,-0.002576,-0.000821,-0.000248,0.000149,-0.003528,-0.012457,...,-0.008272,-0.006614,0.000083,-0.002788,0.000374,-0.000138,-0.002683,-0.001586,0,d+10


- 정답 양식으로 변경

In [ ]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [ ]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.241222,-0.276154,-0.277583,-0.194854,-0.159025,-0.137008,0.009250,-0.202066,...,-0.196708,-0.370782,-0.253162,-0.221149,0.000016,-0.258488,-0.199812,-0.186842,-0.230400,-0.260928
1,0,d+2,-0.382954,-0.390344,-0.393883,-0.313928,-0.252214,-0.178264,0.006073,-0.341174,...,-0.227133,-0.579484,-0.400144,-0.337676,-0.000008,-0.394654,-0.271525,-0.234227,-0.354214,-0.374773
2,0,d+3,-0.344161,-0.372666,-0.352539,-0.287831,-0.235053,-0.175451,0.004323,-0.362716,...,-0.220710,-0.511771,-0.330968,-0.300342,-0.000025,-0.376702,-0.278547,-0.229778,-0.329226,-0.335397
3,0,d+4,-0.331768,-0.355947,-0.291278,-0.267505,-0.227120,-0.170433,0.004206,-0.339939,...,-0.222216,-0.439430,-0.317751,-0.261915,0.000042,-0.381903,-0.265135,-0.222977,-0.325889,-0.317033
4,0,d+5,-0.347585,-0.338480,-0.268488,-0.266670,-0.233057,-0.164194,0.004160,-0.302935,...,-0.224520,-0.421779,-0.349862,-0.239883,0.000001,-0.400415,-0.259995,-0.208940,-0.341349,-0.344145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.447310,-0.101804,-0.122182,-0.987412,-1.510957,-0.030516,-0.037136,NaN,...,-0.036214,-0.079450,-0.075609,-0.065056,-0.000050,-0.075238,0.013158,-23.003152,-0.070007,-0.101035
146,9,d+12,0.445692,-0.095712,-0.103264,-0.996746,-1.552245,-0.026432,-0.040630,NaN,...,-0.044028,-0.068355,-0.002104,-0.016428,0.000011,-0.071664,0.035473,-20.094368,-0.071358,-0.100675
147,9,d+13,0.284697,-0.059149,-0.066080,-0.629728,-0.940218,-0.017253,-0.050715,NaN,...,-0.030830,-0.042255,-0.024917,-0.015700,-0.000014,-0.047558,0.062727,-12.954190,-0.048924,-0.071235
148,9,d+14,0.037463,-0.008788,-0.000791,-0.124893,-0.153622,0.000466,-0.033125,NaN,...,-0.007241,-0.018096,0.018505,0.006780,-0.000024,-0.004167,0.075813,-3.574751,-0.008264,-0.002027


- 제출 양식

In [ ]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [ ]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.003206,-0.005402,-0.004850,-0.003621,-0.003263,-0.001663,-0.000315,-0.003737,...,-0.001965,-0.010341,-0.007796,-0.008307,0.000006,-0.002952,-0.001875,-0.002722,-0.003897,-0.006194
1,0,d+2,-0.002290,-0.005933,-0.004775,-0.003966,-0.003514,-0.001338,NaN,-0.003304,...,-0.001128,-0.013213,-0.008756,-0.009856,-0.000061,-0.002158,-0.001093,-0.001868,-0.003034,-0.004661
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,-0.000213,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,-0.001707,-0.002963,-0.002560,-0.002330,-0.002160,-0.000771,NaN,-0.001482,...,-0.000530,-0.009648,-0.005431,-0.005148,-0.000062,-0.002424,-0.000102,-0.000578,-0.001963,-0.000562
4,0,d+5,0.000097,-0.000524,-0.000100,-0.001118,-0.000891,-0.000309,NaN,-0.000085,...,-0.000201,-0.005397,-0.001626,0.000105,0.000057,-0.001904,0.000648,0.000138,-0.000700,0.002321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.004448,-0.001149,0.000201,-0.007728,-0.010854,-0.000507,-0.000964,NaN,...,-0.000647,-0.008531,0.002618,0.002124,-0.000033,-0.002641,0.018524,-0.083729,-0.002258,0.002757
146,9,d+12,0.005481,-0.000332,0.002697,-0.008349,-0.014100,-0.000283,-0.000591,NaN,...,-0.000285,-0.006577,0.011998,0.005407,-0.000096,-0.001297,0.020676,-0.065204,-0.001372,0.004683
147,9,d+13,0.005577,-0.000620,0.002234,-0.007376,-0.013782,-0.000197,-0.000773,NaN,...,-0.000190,-0.007189,0.009053,0.004735,0.000035,-0.001130,0.024579,-0.025186,-0.001361,0.004455
148,9,d+14,0.006801,-0.000703,0.001179,-0.003990,-0.009245,-0.000055,-0.000640,NaN,...,-0.000030,-0.006664,0.001331,0.004689,-0.000002,-0.001266,0.027517,0.000917,-0.001122,0.004550
